# OpenClaw on Google Colab (Free CPU)

Run your own **OpenClaw** AI gateway entirely on a free Google Colab CPU instance.

| Step | What | Time |
|------|------|------|
| **1** | Install Node.js 22 | ~30s |
| **2** | Install OpenClaw from npm | ~30s |
| **3** | Enter your API key | ~10s |
| **4** | Start the gateway | ~5s |

**Just run each cell top-to-bottom. That's it.**

> Data is stored locally in this Colab session. If the runtime recycles, just re-run the cells.
> For persistent storage across sessions, see the optional Google Drive step at the bottom.

---
## Step 1: Install Node.js 22

OpenClaw requires Node.js >= 22. Colab ships an older version, so we install a fresh one. npm comes bundled with it.

In [ ]:
%%bash
set -euo pipefail

# Check if Node 22+ is already installed (from a previous run in this session)
if command -v node &>/dev/null; then
  NODE_MAJOR=$(node -v | sed 's/v\([0-9]*\).*/\1/')
  if [ "$NODE_MAJOR" -ge 22 ]; then
    echo "Node.js $(node -v) already installed, skipping."
    echo "npm $(npm -v)"
    exit 0
  fi
fi

echo "Installing Node.js 22 via NodeSource..."
curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash - 2>&1 | tail -3
sudo apt-get install -y nodejs 2>&1 | tail -3

echo
echo "Node.js $(node -v)"
echo "npm $(npm -v)"

---
## Step 2: Install OpenClaw

Installs the pre-built OpenClaw package from npm. No cloning, no compiling — takes under a minute.

In [ ]:
%%bash
set -euo pipefail

# Check if already installed
if command -v openclaw &>/dev/null; then
  echo "OpenClaw already installed: $(openclaw --version 2>/dev/null || echo 'unknown version')"
  echo "To upgrade, run: npm update -g openclaw"
else
  echo "Installing OpenClaw from npm..."
  npm install -g openclaw@latest 2>&1 | tail -5
  echo
  echo "Installed: $(openclaw --version 2>/dev/null || echo 'done')"
fi

echo
echo "OpenClaw location: $(which openclaw)"

---
## Step 3: Provide Your API Key(s)

Enter at least one LLM provider API key. Press Enter to skip any provider you don't use.

Supported: **Anthropic**, **OpenAI**, **Gemini**, **OpenRouter**

In [ ]:
import getpass
import os
import secrets

# State directory (local by default, no Drive permissions needed)
OPENCLAW_STATE = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
os.makedirs(OPENCLAW_STATE, exist_ok=True)

print('Enter your LLM API key(s). Press Enter to skip a provider.\n')

keys = {}

anthropic_key = getpass.getpass('Anthropic API key (sk-ant-...): ')
if anthropic_key.strip():
    keys['ANTHROPIC_API_KEY'] = anthropic_key.strip()

openai_key = getpass.getpass('OpenAI API key (sk-...): ')
if openai_key.strip():
    keys['OPENAI_API_KEY'] = openai_key.strip()

gemini_key = getpass.getpass('Gemini API key: ')
if gemini_key.strip():
    keys['GEMINI_API_KEY'] = gemini_key.strip()

openrouter_key = getpass.getpass('OpenRouter API key (sk-or-...): ')
if openrouter_key.strip():
    keys['OPENROUTER_API_KEY'] = openrouter_key.strip()

if not keys:
    print('\n\u26a0\ufe0f  No API keys provided! OpenClaw needs at least one to function.')
    print('Re-run this cell to enter your key(s).')
else:
    for k, v in keys.items():
        os.environ[k] = v
    print(f'\n\u2705 {len(keys)} API key(s) configured: {", ".join(keys.keys())}')

# Generate a gateway token for this session
gateway_token = secrets.token_hex(32)
os.environ['OPENCLAW_GATEWAY_TOKEN'] = gateway_token
os.environ['OPENCLAW_STATE_DIR'] = OPENCLAW_STATE

# Write .env so the gateway process picks it up
env_path = os.path.join(OPENCLAW_STATE, '.env')
with open(env_path, 'w') as f:
    f.write(f'OPENCLAW_GATEWAY_TOKEN={gateway_token}\n')
    for k, v in keys.items():
        f.write(f'{k}={v}\n')

print(f'\nGateway token generated (for auth when connecting remotely).')
print(f'State directory: {OPENCLAW_STATE}')

---
## Step 4: Start OpenClaw Gateway

Launches the gateway as a background process. The dashboard and API will be available on port 18789.

In [ ]:
import subprocess
import os
import time

OPENCLAW_STATE = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')

# Build the environment
env = os.environ.copy()
env['OPENCLAW_STATE_DIR'] = OPENCLAW_STATE
env['HOME'] = '/content'
env['NODE_ENV'] = 'production'
env['OPENCLAW_SKIP_CHANNELS'] = '1'

# Symlink state dir to ~/.openclaw so the gateway finds it naturally
home_openclaw = '/content/.openclaw'
if os.path.islink(home_openclaw):
    os.unlink(home_openclaw)
if not os.path.exists(home_openclaw):
    os.symlink(OPENCLAW_STATE, home_openclaw)

# Kill any previous gateway process
try:
    subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
    time.sleep(2)
except Exception:
    pass

# Find the openclaw binary
openclaw_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()
if not openclaw_bin:
    print('\u274c OpenClaw not found. Please run Step 2 first.')
else:
    # Start the gateway
    log_file = open('/content/openclaw_gateway.log', 'w')
    proc = subprocess.Popen(
        [openclaw_bin, 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
        env=env,
        stdout=log_file,
        stderr=subprocess.STDOUT,
        preexec_fn=os.setsid
    )

    print(f'Gateway starting (PID: {proc.pid})...')
    time.sleep(5)

    # Check if it's still running
    if proc.poll() is None:
        print('\n\u2705 OpenClaw Gateway is running!')
        print(f'   Local URL: http://localhost:18789')
        print(f'   Logs:      /content/openclaw_gateway.log')
        print(f'   State:     {OPENCLAW_STATE}')
        print(f'\n   Gateway Token: {os.environ.get("OPENCLAW_GATEWAY_TOKEN", "N/A")[:12]}...')
    else:
        print('\n\u274c Gateway failed to start. Logs:')
        with open('/content/openclaw_gateway.log') as f:
            print(f.read()[-3000:])

---
## Optional: Expose via ngrok

Access OpenClaw from outside Colab (connect messaging channels, use from your phone, etc.).

1. Get a free auth token at https://ngrok.com
2. Paste it below

In [ ]:
import getpass

ngrok_token = getpass.getpass('ngrok auth token (or press Enter to skip): ')

if ngrok_token.strip():
    import subprocess
    subprocess.run(['pip', 'install', 'pyngrok', '-q'], check=True)

    from pyngrok import ngrok, conf
    conf.get_default().auth_token = ngrok_token.strip()

    tunnel = ngrok.connect(18789, 'http')
    print(f'\n\u2705 ngrok tunnel active!')
    print(f'   Public URL: {tunnel.public_url}')
    print(f'\n   Use this URL to access your OpenClaw gateway from anywhere.')
    print(f'   You\'ll need your Gateway Token to authenticate.')
else:
    print('Skipped ngrok. Gateway is only accessible within this Colab session.')
    print('You can use the Gradio UI below to interact locally.')

---
## Optional: Chat UI (Gradio)

A lightweight chat interface right here in the notebook.

In [ ]:
!pip install gradio requests -q

import gradio as gr
import requests
import os

GATEWAY_URL = 'http://localhost:18789'
TOKEN = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')


def check_gateway_health():
    try:
        r = requests.get(f'{GATEWAY_URL}/health', timeout=5)
        return r.status_code == 200
    except Exception:
        return False


def send_message(message, history):
    if not check_gateway_health():
        return 'Gateway is not running. Please run Step 4 first.'
    try:
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {TOKEN}',
        }
        payload = {'message': message, 'channel': 'webchat'}
        r = requests.post(
            f'{GATEWAY_URL}/api/chat',
            headers=headers,
            json=payload,
            timeout=120,
        )
        if r.status_code == 200:
            data = r.json()
            return data.get('reply', data.get('message', str(data)))
        else:
            return f'Error {r.status_code}: {r.text[:500]}'
    except requests.exceptions.Timeout:
        return 'Request timed out. The model may be processing a complex query.'
    except Exception as e:
        return f'Error: {str(e)}'


with gr.Blocks(title='OpenClaw Chat') as demo:
    gr.Markdown('## OpenClaw Chat')
    gr.Markdown('Chat with your OpenClaw gateway directly from the notebook.')
    chatbot = gr.ChatInterface(
        fn=send_message,
        type='messages',
        examples=[
            'Hello! What can you do?',
            'What model are you using?',
            'Summarize the latest news about AI.',
        ],
    )

demo.launch(share=False, debug=False)

---
## Optional: Google Drive Persistence

By default, data lives in Colab's local filesystem and is lost when the runtime recycles. If you want data to survive between sessions, run this cell to mount Google Drive and move the state there.

**Note:** This will request full Google Drive access. Only use this if you need persistence across sessions.

In [ ]:
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')

DRIVE_STATE = '/content/drive/MyDrive/openclaw/state'
LOCAL_STATE = '/content/openclaw_state'

os.makedirs(DRIVE_STATE, exist_ok=True)

# Copy any existing local state to Drive
if os.path.exists(LOCAL_STATE):
    for item in os.listdir(LOCAL_STATE):
        src = os.path.join(LOCAL_STATE, item)
        dst = os.path.join(DRIVE_STATE, item)
        if os.path.isfile(src):
            shutil.copy2(src, dst)
        elif os.path.isdir(src) and not os.path.exists(dst):
            shutil.copytree(src, dst)
    print(f'Copied local state to Google Drive.')

# Update environment so next gateway start uses Drive
os.environ['OPENCLAW_STATE_DIR'] = DRIVE_STATE

# Update symlink
home_openclaw = '/content/.openclaw'
if os.path.islink(home_openclaw):
    os.unlink(home_openclaw)
os.symlink(DRIVE_STATE, home_openclaw)

print(f'\n\u2705 Google Drive persistence enabled.')
print(f'   State directory: {DRIVE_STATE}')
print(f'\n   Restart the gateway (re-run Step 4) to use Drive storage.')

---
## Utilities

In [ ]:
# View recent gateway logs
!tail -50 /content/openclaw_gateway.log

In [ ]:
# Check if gateway is still running
!pgrep -fa 'openclaw.*gateway' || echo 'Gateway is NOT running. Re-run Step 4.'

In [ ]:
# Stop the gateway
!pkill -f 'openclaw.*gateway' && echo 'Gateway stopped.' || echo 'No gateway process found.'

In [ ]:
# Check state directory contents
import os
state_dir = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
print(f'State directory: {state_dir}\n')
for root, dirs, files in os.walk(state_dir):
    level = root.replace(state_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    if level < 3:
        subindent = ' ' * 2 * (level + 1)
        for file in files[:20]:
            size = os.path.getsize(os.path.join(root, file))
            print(f'{subindent}{file} ({size:,} bytes)')